In [1]:
!pip install h3
!pip install PyAstronomy

You should consider upgrading via the '/home/nes/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/nes/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install folium

You should consider upgrading via the '/home/nes/anaconda3/bin/python -m pip install --upgrade pip' command.


In [29]:
!pip install chromedriver-binary==89.0.4389.23.0

  Created wheel for chromedriver-binary: filename=chromedriver_binary-89.0.4389.23.0-py3-none-any.whl size=5852118 sha256=76f08dbfac00272745abf4f561ec1066eb320a8a41401387e03fd91501368b73
  Stored in directory: /home/nes/.cache/pip/wheels/15/d9/55/4893ad90daf4e73ba7199f08eda0658bf28108ab7f00123a4a
Successfully built chromedriver-binary
You should consider upgrading via the '/home/nes/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
import os
import time
from selenium import webdriver
import chromedriver_binary
from h3 import h3
import folium

In [2]:
# taken and modified from https://github.com/uber/h3-py-notebooks/blob/master/notebooks/usage.ipynb June 15th, 2021

def filter_coordinates(lat, lon):
    if (lon > -95.776056) and (lon < 105.662588) and (lat < 67.107210) and (lat > -20.606050):
        return True
    else:
        return False
    
def filter_keygroups(keygroup_hex):
    lat, lon = h3.h3_to_geo(keygroup_hex)
    # 62.107210, -52.325859
    # 9.180394, -52.325859
    # 62.107210, 85.985628
    # 9.180394, 85.985628
    return filter_coordinates(lat, lon)
    


def visualize_hexagons(hexagons, color="black", folium_map=None, color_map=None, fill=True, weight=3, opacity=0.1):
    """
    https://nbviewer.jupyter.org/github/uber/h3-py-notebooks/blob/master/notebooks/usage.ipynb
    
    this one is just for testing purposes
    
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    # polylines = []
    lat = []
    lng = []
    
    if folium_map is None:
        m = folium.Map(location=[31.5194043564895, 15.158257565930684], zoom_start=3, tiles='cartodbpositron', 
                       min_lat=9.180394, max_lat=62.107210, min_lon=-52.325859, max_lon=85.985628,)
    else:
        m = folium_map
    for hex in hexagons:
        if filter_keygroups(hex):
            polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
            # flatten polygons into loops.
            outlines = [loop for polygon in polygons for loop in polygon]
            polyline = [outline + [outline[0]] for outline in outlines][0]
            lat.extend(map(lambda v:v[0],polyline))
            lng.extend(map(lambda v:v[1],polyline))
            # polylines.append(polyline)
            if color_map is not None:
                color = color_map[hex]
            my_PolyLine=folium.PolyLine(locations=polyline,opacity=opacity, fill=fill, weight=weight,color=color)
            m.add_child(my_PolyLine)        
    return m

def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m

def visualize_ground_stations(folium_map, ground_stations):
    m = folium_map
    for gst in ground_stations:
            folium.CircleMarker(location=ground_stations[gst], radius=6, color="red").add_to(m)
    return m
    
def visualize_keygroups_and_satellites(satellite_and_keygroup, folium_map, color_map=None):
       
    m = folium_map
    for key in satellite_and_keygroup:
        keygroup = [satellite_and_keygroup[key]["keygroup"]]
        color = "black"
        for hex in keygroup:
            if filter_keygroups(hex):
                
                #print(color_map[hex])
                polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
                # flatten polygons into loops.
                outlines = [loop for polygon in polygons for loop in polygon]
                polyline = [outline + [outline[0]] for outline in outlines][0]
                
                if color_map is not None:
                    color = color_map[hex]
                else: 
                    color = satellites[key]["color"]
                #my_PolyLine=folium.PolyLine(locations=polyline,smooth_factor=3,no_clip=True, weight=3,color=satellites[key]["color"])
                #m.add_child(my_PolyLine)
        #print(color)
        for coord in satellite_and_keygroup[key]["coord"]:
            if filter_coordinates(coord[0], coord[1]):
                folium.CircleMarker(location=coord, radius=3, color=color).add_to(m)
    
    return m

def visualize_gst_sat_connection(folium_map, gst_loc, sat_loc):
    m = folium_map
    folium.PolyLine(
        locations = [gst_loc, sat_loc], # tuple of coordinates 
        color="red",
        weight=3
        ).add_to(m)
    return m
    

In [9]:
# This file is part of the uni project "Distributed Systems Prototyping - LEO-CDN" at TU Berlin.
# Copyright (c) 2020 Ben S. Kempton, LEO-CDN project group.
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, version 3.
#
# This program is distributed in the hope that it will be useful, but
# WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU
# General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program. If not, see <http://www.gnu.org/licenses/>.
import json
import math
import toml
import numpy as np
from PyAstronomy import pyasl

EARTH_RADIUS = 6371000  # in meter
ALTITUDE = 550  # Orbit Altitude (Km)

# according to wikipedia
STD_GRAVITATIONAL_PARAMETER_EARTH = 3.986004418e14
# seconds the earth needs to make one whole rotation
SECONDS_PER_DAY = 60 * 60 * 24
# earth's z axis (eg a vector in the positive z direction)
EARTH_ROTATION_AXIS = [0, 0, 1]  # this means the north pole is on the top and the south pole is on the bottom
semi_major_axis = float(ALTITUDE) * 1000 + EARTH_RADIUS


class Constellation:
    """
    A class used to contain and manage a satellite constellation
    ...
    Attributes
    ----------
    number_of_planes : int
        the number of planes in the constellation
    nodes_per_plane : int
        the number of satellites per plane
    semi_major_axis : float
        semi major axis of the orbits (radius, if orbits circular)
    period : int
        the period of the orbits in seconds
    list_of_satellites: List[Satellites]
        list of all the satellites in the system.
    current_time : int
        keeps track of the current simulation time

    Methods
    -------
    init_satellites()
        Creates kepler ellipse for each satellites and initialize its position at time 0.
    update_position(time=0)
        Updates all satellites to reflect the new time.
    get_all_satellites()
        Returns all the satellites as a list.
        Each satellite contains information like position (x, y, z) and keygroup.
    print_current_position()
        Prints all the position of each satellite.
    print_current_keygroups()
        Print all satellites and their keygroup they currently belong to.
    """

    def __init__(self,
                 number_of_planes,
                 nodes_per_plane,
                 semi_major_axis=SEMI_MAJOR_AXIS):
        self.number_of_planes = number_of_planes
        self.nodes_per_plane = nodes_per_plane
        self.semi_major_axis = semi_major_axis
        self.period = self.calculate_orbit_period(semi_major_axis=self.semi_major_axis)
        self.list_of_satellites = list()
        self.dict_of_ground_stations = dict()
        self.current_time = 0

        self.init_satellites()

    def init_satellites(self):
        # figure out how many degrees to space right ascending nodes of the planes
        raan_offsets = [(360 / self.number_of_planes) * i for i in range(0, self.number_of_planes)]

        # at which time of the kepler ellipse is the satellite
        # like degree offset (raan_offsets) but as a time in seconds
        time_offsets = [(self.period / self.nodes_per_plane) * i for i in range(0, self.nodes_per_plane)]

        # we offset each plane by a small amount, so they do not "collide"
        # this little algorithm comes up with a list of offset values
        phase_offset = 0
        phase_offset_increment = (self.period / self.nodes_per_plane) / self.number_of_planes
        temp = []
        toggle = False
        # this loop results puts thing in an array in this order:
        # [...8,6,4,2,0,1,3,5,7...]
        # so that the offsets in adjacent planes are similar
        # basically do not want the max and min offset in two adjcent planes
        for i in range(self.number_of_planes):
            if toggle:
                temp.append(phase_offset)
            else:
                temp.insert(0, phase_offset)
                # temp.append(phase_offset)
            toggle = not toggle
            phase_offset = phase_offset + phase_offset_increment

        phase_offsets = temp

        # create kepler ellipse for each degree offset
        list_of_kepler_ellipse = list()
        for raan in raan_offsets:
            ellipse = pyasl.KeplerEllipse(
                per=self.period,  # how long the orbit takes in seconds
                a=self.semi_major_axis,  # if circular orbit, this is same as radius
                e=0,  # generally close to 0 for leo constillations
                Omega=raan,  # right ascention of the ascending node
                w=0.0,  # initial time offset / mean anamoly
                i=53,  # orbit inclination
            )
            list_of_kepler_ellipse.append(ellipse)

        for plane in range(0, self.number_of_planes):
            for node in range(0, self.nodes_per_plane):
                ellipse = list_of_kepler_ellipse[plane]
                # calculate the KE solver time offset
                offset = time_offsets[node] + phase_offsets[plane]
                satellite_number = (plane * self.nodes_per_plane) + node
                new_satellite = Satellite(
                    name=f"satellite{satellite_number}",
                    number=satellite_number,
                    kepler_ellipse=ellipse,
                    offset=offset,
                )
                self.list_of_satellites.append(new_satellite)

    def add_new_ground_station(self, ground_station_id, lat, lon):
        x, y, z = self.transform_geo_to_xyz(lat, lon)
        self.dict_of_ground_stations[ground_station_id] = {"x": x, "y": y, "z": z, "lat": lat, "lon": lon}

    def add_new_ground_station_xyz(self, ground_station_id, x, y, z):
        self.dict_of_ground_stations[ground_station_id] = {"x": x, "y": y, "z": z}

    def transform_geo_to_xyz(self, lat, lon):
        """
        Transforms any latitude and longitude to xyz coordinates.
        Taken from Ben S. Kempton.
        Parameters
        ----------
        lat
        lon

        Returns
        -------

        """
        # must convert the lat/long/alt to cartesian coordinates
        radius = EARTH_RADIUS + ALTITUDE
        init_pos = [0, 0, 0]
        latitude = math.radians(lat)
        longitude = math.radians(lon)
        init_pos[0] = radius * math.cos(latitude) * math.cos(longitude)  # x
        init_pos[1] = radius * math.cos(latitude) * math.sin(longitude)  # y
        init_pos[2] = radius * math.sin(latitude)  # z

        # if simulation time is not 0, figure out current position
        if self.current_time == 0 or self.current_time % SECONDS_PER_DAY == 0:
            pos = init_pos

        else:
            degrees_to_rotate = 360.0 / (SECONDS_PER_DAY / (self.current_time % SECONDS_PER_DAY))
            rotation_matrix = self.get_rotation_matrix(EARTH_ROTATION_AXIS, degrees_to_rotate)

            pos = np.dot(rotation_matrix, init_pos)

        x = np.int32(pos[0])
        y = np.int32(pos[1])
        z = np.int32(pos[2])

        return x, y, z

    def get_rotation_matrix(self, axis, degrees):
        """
        Return the rotation matrix associated with counterclockwise rotation about
        the given axis by theta radians.

        Taken from Ben S. Kempton.
        Parameters
        ----------
        axis : list[x, y, z]
            a vector defining the rotaion axis
        degrees : float
            The number of degrees to rotate
        """

        theta = math.radians(degrees)
        axis = np.asarray(axis)
        axis = axis / math.sqrt(np.dot(axis, axis))
        a = math.cos(theta / 2.0)
        b, c, d = -axis * math.sin(theta / 2.0)
        aa, bb, cc, dd = a * a, b * b, c * c, d * d
        bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
        return np.array([
            [aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
            [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
            [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

    def calculate_orbit_period(self, semi_major_axis=0.0):
        """
        Calculates the period of a orbit for Earth.

        Parameters
        ----------
        semi_major_axis : float
            semi major axis of the orbit in meters

        Returns
        -------
        Period : int
            the period of the orbit in seconds (rounded to whole seconds)
        """

        tmp = math.pow(semi_major_axis, 3) / STD_GRAVITATIONAL_PARAMETER_EARTH
        return int(2.0 * math.pi * math.sqrt(tmp))

    def update_position(self, time=0):
        """
        Updates the positions of the satellites.

        Parameters
        ----------
        time :
            The new current time.

        """
        self.current_time = time

        for satellite in self.list_of_satellites:
            satellite.set_new_position(self.current_time)

    def get_best_satellite(self, ground_station_id):
        """
        Calculates which of the satellites is the nearest to a ground stations and returns this nearest satellite.
        Parameters
        ----------
        ground_station_id

        Returns
        -------

        """
        ground_station = self.dict_of_ground_stations[ground_station_id]
        shortest_distance = np.inf
        best_satellite = None
        for satellite in self.list_of_satellites:
            x, y, z = satellite.get_current_position()
            # calculate euclidean distance
            distance = int(math.sqrt(
                math.pow(x - ground_station["x"], 2) +
                math.pow(y - ground_station["y"], 2) +
                math.pow(z - ground_station["z"], 2)))
            if distance < shortest_distance:
                # TODO check whether the distance is even allowed
                shortest_distance = distance
                best_satellite = satellite
        return best_satellite.get_current_geo_position()

    def print_current_position(self):
        """
        Prints the current position of every satellite.
        This position is however without any rotation in case we want to simulate the ground station.
        (For the ground stations we would apply the rotation on the ground stations instead of the satellites.)
        Returns
        -------

        """
        for sat in self.list_of_satellites:
            print(f"{sat.name}")
            print(sat.get_current_position())

    def print_current_keygroups(self):
        """
        Prints the current keygroups of the satellites.
        Returns
        -------

        """
        for sat in self.list_of_satellites:
            print(f"{sat.name} is in {sat.keygroup}")

    def get_all_satellites(self):
        """
        Returns the list of all satellites.
        Returns
        -------

        """
        return self.list_of_satellites

    def get_satellite(self, satellite_id):
        """
        Returns a specific satellite.
        Parameters
        ----------
        satellite_id

        Returns
        -------

        """
        satellite = None
        for sat in self.list_of_satellites:
            if sat.name == satellite_id:
                satellite = sat
        if satellite is None:
            raise NoSuchSatelliteError
        else:
            return satellite

    def get_satellite_position(self, satellite_id):
        """
        Returns the x,y,z coordinates of a satellite.
        Parameters
        ----------
        satellite_id

        Returns
        -------

        """
        sat = self.get_satellite(satellite_id)
        return sat.get_current_position()

    def get_satellite_keygroup(self, satellite_id):
        """
        Returns the Keygroup in which the satellite is at the moment.
        Parameters
        ----------
        satellite_id

        Returns
        -------

        """
        sat = self.get_satellite(satellite_id)
        return sat.keygroup

    def get_ground_station_position(self, ground_station_id):
        """
        Returns the position of a ground station.
        Parameters
        ----------
        ground_station_id

        Returns
        -------

        """
        ground_station = self.dict_of_ground_stations[ground_station_id]
        return ground_station["x"], ground_station["y"], ground_station["z"]

    def get_ground_station_geo_position(self, ground_station_id):
        """
        Returns the geographical position (longitude and latitude) of a ground station.
        Parameters
        ----------
        ground_station_id

        Returns
        -------

        """
        ground_station = self.dict_of_ground_stations[ground_station_id]
        return ground_station["lat"], ground_station["lon"]

In [10]:
# This file is part of the uni project "Distributed Systems Prototyping - LEO-CDN" at TU Berlin.
# Copyright (c) 2020 Ben S. Kempton, LEO-CDN project group.
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, version 3.
#
# This program is distributed in the hope that it will be useful, but
# WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU
# General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program. If not, see <http://www.gnu.org/licenses/>.

import math
import numpy as np
from h3 import h3


class Satellite:
    """
    This class holds all the information about a single satellite.
    Kepler Ellipse, offset are needed for our own simulation. For the integration into Celestial these two are not
    needed hence, we do not need to use e.g. update position in Celestial.


    Attributes
    ----------
    name: str
        Name of the satellite as "satellite{id} of node on plane}"
    kepler_ellipse: KeplerEllipse
        The kepler ellipse of this satellite.
    offset: int
        The offset of this satellite
    current_time: int
        The current time of the simulation in seconds.
    x_position: int
        The unrotated x position of the satellite.
    y_position: int
        The unrotated y position of the satellite.
    z_position: int
        The unrotated z position of the satellite.
    keygroup:
        The current keygroup of the satellite.
    host:
        Host of the satellite.
    port:
        Port of the satellite.

    """

    def __init__(self, name, number, kepler_ellipse=None, offset=0):
        self.name = name
        self.number = number
        self.kepler_ellipse = kepler_ellipse
        self.offset = offset
        self.current_time = 0
        current_position = self.kepler_ellipse.xyzPos(self.offset)
        self.x_position = current_position[0]
        self.y_position = current_position[1]
        self.z_position = current_position[2]
        self.keygroup = self.init_keygroup()

    def set_new_position(self, current_time):
        """
        Sets the new position of a satellite.
        It calls xyzPos from PyAstronomy KeplerEllipse. The parameter for this function is the time.
        Therefore, we have to add the current time to the offset (the initial position of the satellite)

        Parameters
        ----------
        current_time : int
            new time to calculate the new position

        Returns
        -------

        """
        self.current_time = current_time

        updated_position = self.kepler_ellipse.xyzPos(current_time + self.offset)
        self.x_position = np.int32(updated_position[0])
        self.y_position = np.int32(updated_position[1])
        self.z_position = np.int32(updated_position[2])

        self.check_keygroup()

    def set_xyz_position(self, x, y, z):
        self.x_position = x
        self.y_position = y
        self.z_position = z

    def get_current_position(self):
        """
        Returns the current position of a satellite but without the rotation

        Returns
        -------
        x_position: int
        y_position: int
        z_position: int
        """
        return self.x_position, self.y_position, self.z_position

    def get_rotation_matrix(self, axis, degrees):
        """
        Return the rotation matrix associated with counterclockwise rotation about
        the given axis by theta radians.

        This function is copied from Ben S. Kempton and has not been modified.

        Parameters
        ----------
        axis : list[x, y, z]
            a vector defining the rotaion axis
        degrees : float
            The number of degrees to rotate
        """

        theta = math.radians(degrees)
        axis = np.asarray(axis)
        axis = axis / math.sqrt(np.dot(axis, axis))
        a = math.cos(theta / 2.0)
        b, c, d = -axis * math.sin(theta / 2.0)
        aa, bb, cc, dd = a * a, b * b, c * c, d * d
        bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
        return np.array([
            [aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
            [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
            [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

    def get_current_geo_position(self):
        """
        This formula considers the earth to be an ellipsoid.
        It does not calculate the altitude since we only want to know where the position on earth is.
        Returns
        -------
        lon:
            longitude of the position of the satellite
        lat:
            latitude of the position of the satellite

        """
        seconds_per_day = 60 * 60 * 24
        # earth's z axis (eg a vector in the positive z direction)
        earth_rotation_axis = [0, 0, 1]  # this means the north pole is on the top and the south pole is on the bottom

        if self.current_time == 0 or self.current_time % seconds_per_day == 0:
            degrees_to_rotate = 0
        else:
            # how many degrees from time 0 should the earth rotate?
            degrees_to_rotate = 360.0 / (seconds_per_day / (self.current_time % seconds_per_day))

        # have to -degrees to rotate because the satellites moved too far away from the current
        # keygroup and we cannot move the keygroup position because the satellites determine in
        # which keygroup they belong to
        rotation_matrix = self.get_rotation_matrix(earth_rotation_axis, -degrees_to_rotate)

        current_position = self.get_current_position()
        rotated_position = np.dot(rotation_matrix, current_position)

        rotated_x = rotated_position[0]
        rotated_y = rotated_position[1]
        rotated_z = rotated_position[2]

        radius = 6371000
        lon = np.degrees(np.arctan2(rotated_y, rotated_x))
        lat = np.degrees(np.arcsin(rotated_z / radius))

        return lat, lon

    def init_keygroup(self, hex_resolution=0):
        """
        Sets the satellite in the correct initial keygroup.

        Parameters
        ----------
        hex_resolution: int
            Can be between 0 and 15. Determines the size of the hexagon.

        Returns
        -------

        """
        lat, lon = self.get_current_geo_position()
        keygroup_name = h3.geo_to_h3(lat, lon, hex_resolution)  # is the same as h3 area

        # print(f"[satellites.py]: Initializing keygroup {keygroup_name} at node {self.name}...")
        # target_node = self.fred

        # status_response = keygroup_passer.create_keygroup(target_node=target_node, keygroup=keygroup_name)

        # status_response = add_replica_node_to_keygroup(target_node=target_node, keygroup=keygroup_name)

        # if status_response.status == 1 or status_response == 2:
            # print(f"Oh no. Something went wrong.")
            # print(status_response.message)
        return keygroup_name

    def check_keygroup(self, hex_resolution=0):
        """
        Checks whether the keygroup is still the same.
        If yes it returns 0. Otherwise, it returns the new keygroup_id.

        Parameters
        ----------
        hex_resolution

        Returns
        -------

        """
        lat, lon = self.get_current_geo_position()
        new_keygroup_name = h3.geo_to_h3(lat, lon, hex_resolution)  # is the same as h3 area
        old_keygroup_name = self.keygroup
        if new_keygroup_name == old_keygroup_name:
            return None
        else:
            # target_node = self.fred
            # print(f"Changing keygroup for {self.name} from {old_keygroup_name} to {new_keygroup_name}...")
            # status_response_add = add_replica_node_to_keygroup(target_node=target_node,
            #                                                                    keygroup=f"{new_keygroup_name}")
            # If adding to a keygroup does not work out create the keygroup.
            # print(f"[satellite.py]: Status response: {status_response_add}")
            # if status_response_add.status == 1 or status_response_add.status == 2:
            # print("Cannot add to keygroup.")
            # print(status_response_add.message)

            # Removing satellite from keygroup
            # status_response_remove = remove_replica_node_from_keygroup(target_node=target_node,
            #                                                                            keygroup=f"{old_keygroup_name}")

            # print(f"[satellite.py]: Status response: {status_response_remove}")
            # If the satellite cannot be removed from the keygroup
            # if status_response_remove.status == 1:
            #     print("Cannot remove to keygroup.")
            #    print(status_response_remove.message)
            self.keygroup = new_keygroup_name
            return new_keygroup_name

In [14]:
number_of_planes = 10
nodes_per_plane = 18
ground_stations = {"gst-0": [52.512766, 13.326758], 
                       "gst-1": [48.182779, 11.608218],
                       "gst-2": [52.240076, 21.020137], 
                       "gst-3": [41.888735, 12.500474],
                       "gst-4": [40.693945, -73.995660], 
                       "gst-5": [47.476040, 19.092611],
                      }
constellation = Constellation(number_of_planes=number_of_planes,
                              nodes_per_plane=nodes_per_plane,
                              semi_major_axis=semi_major_axis)

for gst in ground_stations:
    constellation.add_new_ground_station(gst, ground_stations[gst][0], ground_stations[gst][1])
color = [ "green", "lightgreen","yellow", "orange",  "red","pink", "purple", "lightblue", "blue"]

steps = 60
step_length = 10
for step in range(0, steps):
    hex_list= list()
    satellites = dict()
    m_2=None
    h3_address = h3.geo_to_h3(0,0,0) # lat, lng, hex resolution                                                                       
    hex_center_coordinates = h3.h3_to_geo(h3_address) # array of [lat, lng]   
    for i in range(0, 10):
        hex_list.append(list(h3.k_ring_distances(h3_address, 10)[i]))
    hex_list = [item for sublist in hex_list for item in sublist]
    hex_list_as_dict = dict()
    for i in range(0,len(hex_list)):
        hex_list_as_dict[hex_list[i]] = color[i%9]
        
    m_2 = visualize_hexagons(hex_list,folium_map=m_2, color_map=hex_list_as_dict)
        
    m_2 = visualize_ground_stations(m_2, ground_stations)
    
    next_time = step * step_length

    constellation.update_position(time=next_time)

    all_satellites = constellation.get_all_satellites()

    for sat in all_satellites:
        lat, lon = sat.get_current_geo_position()
        if not sat.name in satellites:
            satellites[sat.name] = {"coord": [], "color": color[sat.number%6], "keygroup": sat.keygroup}
            satellites[sat.name]["coord"] = [[lat, lon]]
        else: 
            satellites[sat.name]["coord"].append([lat, lon])
    m_2 = visualize_keygroups_and_satellites(satellites, m_2, color_map=hex_list_as_dict)
    
    for gst in ground_stations:
        lat, lon = constellation.get_best_satellite(gst)
        m_2 = visualize_gst_sat_connection(m_2, ground_stations[gst], [lat, lon])
    
#display(m_2)
    
    delay=2
    fn=f'plots/html/map{step}.html'
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
    m_2.save(fn)

    browser = webdriver.Chrome()
    browser.get(tmpurl)
    # Give the map tiles some time to load
    time.sleep(delay)
    browser.save_screenshot(f'plots/png/map{step}.png')
    browser.quit()

In [15]:
m=None
hex_list= list()
h3_address = h3.geo_to_h3(0,0,0) # lat, lng, hex resolution                                                                       
hex_center_coordinates = h3.h3_to_geo(h3_address) # array of [lat, lng]   
for i in range(0, 10):
    hex_list.append(list(h3.k_ring_distances(h3_address, 10)[i]))
hex_list = [item for sublist in hex_list for item in sublist]
hex_list_as_dict = dict()
for i in range(0,len(hex_list)):
    hex_list_as_dict[hex_list[i]] = color[i%9]

m = visualize_hexagons(hex_list,folium_map=m, fill=False, weight=4,opacity=0.5)
for i in range(0, 3):
    hex_to_layer_ = h3.geo_to_h3(52.512768, 13.327366, i)
    polygons = list(h3.h3_to_geo_boundary(hex_to_layer_))
    geoJson = {'type': 'Polygon',
     'coordinates': [polygons] }
    h=h3.polyfill(geoJson, i+1)
    m = visualize_hexagons(h,folium_map=m, fill=False, weight=1, opacity=1)
display(m)

delay=10
fn='plots/html/keygroup_hierarchies2.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
m.save(fn)

browser = webdriver.Chrome()
browser.get(tmpurl)
# Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('plots/png/keygroup_hierarchies2.png')
browser.quit()

In [16]:
m=None
hex_list= list()
h3_address = h3.geo_to_h3(52.512778, 13.327175,0) # lat, lng, hex resolution        
h3_address2 = h3.geo_to_h3(52.512778, 13.327175,1)
h3_address3 = h3.geo_to_h3(52.512778, 13.327175,2)
h3_address4 = h3.geo_to_h3(52.512778, 13.327175,3)
hex_center_coordinates = h3.h3_to_geo(h3_address) # array of [lat, lng]   
for i in range(0, 20):
    hex_list.append(list(h3.k_ring_distances(h3_address, 20)[i]))
    hex_list.append(list(h3.k_ring_distances(h3_address2, 20)[i]))
    hex_list.append(list(h3.k_ring_distances(h3_address3, 20)[i]))
    hex_list.append(list(h3.k_ring_distances(h3_address4, 20)[i]))
hex_list = [item for sublist in hex_list for item in sublist]
hex_list_as_dict = dict()

m = visualize_hexagons(hex_list,folium_map=m, fill=False, weight=1,opacity=0.2)

display(m)

delay=10
fn='plots/html/keygroup_hierarchies3.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
m.save(fn)

browser = webdriver.Chrome()
browser.get(tmpurl)
# Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('plots/png/keygroup_hierarchies3.png')
browser.quit()

In [3]:
m = visualize_hexagons(['8025fffffffffff'])
display(m)